In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from bluepysnap import Simulation
from bluepysnap.bbp import Cell

In [ ]:
# Get the simulation
sim = Simulation("/gpfs/bbp.cscs.ch/project/proj42/circuits/CA1.O1/mooc-circuit/simulation_config.json")

In [ ]:
# Get the global spikes report and the specific hippocampus one
spikes = sim.spikes
hippocampus_spikes = spikes["hippocampus_neurons"]

In [ ]:
# get the hippocampus neurons which spiked and check the properties of the spiking neurons
hippocampus_spikes.nodes.get(group=hippocampus_spikes.node_ids).head()

In [ ]:
# filter the report on SP_PC cells from the mc2 column only
# @armando I changed SP_BS to SP_PC because there were no SP_BS spiking
filtered_spikes = spikes.filter(group={Cell.MTYPE: 'SP_PC', 'region': {'$regex': "mc2.*"}})

In [ ]:
filtered_spikes.report.head()

In [ ]:
# Hack to remove with a better simulation
times = filtered_spikes.report.index.values
filtered_spikes.report.loc[times.max() + 0.15] = [3311, "hippocampus_neurons"]
times = filtered_spikes.report.index.values
filtered_spikes.report.loc[times.max() + 0.15] = [602, "hippocampus_neurons"]

In [ ]:
# Spiking neurons and number of spikes per neurons during the simulation for the filtered cells
node_ids, nb_spikes = np.unique(filtered_spikes.report["ids"], return_counts=True)

In [ ]:
# get full set of properties for neurons which spikes more than once 
hippocampus_spikes.nodes.get(group=node_ids[nb_spikes > 1])

In [ ]:
# get full set of  properties for neurons which spikes only once
hippocampus_spikes.nodes.get(group=node_ids[nb_spikes == 1]).head()

In [ ]:
# raster
filtered_spikes.raster()

In [ ]:
# PSTH
filtered_spikes.firing_rate_histogram()

In [ ]:
# ISI distribution
filtered_spikes.isi()

In [ ]:
# ISI frequency distribution 
filtered_spikes.isi(use_frequency=True)

In [ ]:
# get mean frequencies for each neuron from the report
def get_mean_frequencies():
    gb = filtered_spikes.report.groupby(["ids", "population"])
    res = []
    for _, node_spikes in gb:
        time_differences = np.diff(node_spikes.index.to_numpy())
        if len(time_differences) != 0:
            res.append(1000/time_differences.mean())
    return np.asarray(res)

In [ ]:
def plot_spike_frequency(mean_frequencies, ax=None):
    
    if ax is None:
        ax = plt.gca()
        ax.set_xlabel('Frequency [Hz]')
        ax.set_ylabel('# neurons')

    fig = plt.gcf()
    
    mean = mean_frequencies.mean()
    ax.hist(mean_frequencies)
    ax.axvline(x=mean, color='red')
    
    text = f'{mean:2.8f}'
    limits =  ax.get_xlim()
    mean_text_pos = (limits[1] - mean)/(limits[1] - limits[0]) + 0.05
    
    fig.text(mean_text_pos, 0.80, text, color='red', fontsize=12)
    
    return ax

In [ ]:
plot_spike_frequency(get_mean_frequencies())

In [ ]:
from IPython.display import HTML
anim = filtered_spikes.firing_animation()
HTML(anim.to_html5_video())

In [ ]:
# sample traces
#fig, ax = sim.plot.voltage_collage('soma', gids[:5])